In [1]:
import feature_selection_module as fsm
import os
import pandas as pd

from expttools import Experiment

pip install 'aif360[LawSchoolGPA]'
2022-10-29 04:27:27.100932: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 04:27:27.801363: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-29 04:27:28.082448: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /modules/apps/julia/1.7.2/lib
2022-10-29 04:27:28.082498: I t

In [2]:

from expttools import ExperimentResult

In [3]:
class NamedDataset():
    def __init__(self,name,df):
        self.name = name
        self.df = df
        
    def copy(self):
        return self

In [9]:
def name_function(cur_param):
    #print(cur_param)
    name_parts = []
    name_parts.append(cur_param['data'].name) 
    # some short version of features
    name_parts.append(cur_param['technique'].__name__)
    name_parts.append(cur_param['model'].__name__)
    name_parts.append(cur_param['p_att_description'][0])
    
    #print(name_parts)
    return '_'.join(name_parts)

In [5]:

adult_df = pd.read_csv('adult_dataset.csv')
adult_df.drop('Unnamed: 0', axis = 1, inplace = True)
adult = adult_df.copy()
adult['age_bins'] = pd.cut(adult['age'],10)
adult['age_bins'] = adult['age_bins'].astype(str)
adult.head()


,age,education-num,sex,capital-gain,capital-loss,hours-per-week,income-per-year,education,marital-status,native-country,occupation,race,relationship,workclass,age_bins
0,25.0,7.0,1.0,0.0,0.0,40.0,0.0,11th,Never-married,United-States,Machine-op-inspct,Black,Own-child,Private,"(24.3, 31.6]"
1,38.0,9.0,1.0,0.0,0.0,50.0,0.0,HS-grad,Married-civ-spouse,United-States,Farming-fishing,White,Husband,Private,"(31.6, 38.9]"
2,28.0,12.0,1.0,0.0,0.0,40.0,1.0,Assoc-acdm,Married-civ-spouse,United-States,Protective-serv,White,Husband,Local-gov,"(24.3, 31.6]"
3,44.0,10.0,1.0,7688.0,0.0,40.0,1.0,Some-college,Married-civ-spouse,United-States,Machine-op-inspct,Black,Husband,Private,"(38.9, 46.2]"
4,34.0,6.0,1.0,0.0,0.0,30.0,0.0,10th,Never-married,United-States,Other-service,White,Not-in-family,Private,"(31.6, 38.9]"


In [6]:
adult_dataset = NamedDataset('adult',adult)

In [18]:
features_num = ['hours-per-week', 'capital-gain', 'capital-loss','education-num' ] 
features_cat =  ['workclass', 'relationship', 'occupation','education'] 
protected_attributes = ['sex','race','age_bins', 'marital-status']
target = 'income-per-year'

p_att_description = [('sex',[1]),('race',['White']),('age_bins',['(38.9, 46.2]','(46.2, 53.5]','(53.5, 60.8]']),\
                     ('marital-status',['Married-civ-spouse'])]
model = [fsm.base_classifier,fsm.fair_classifier]
technique = [fsm.xy_greater_xa,fsm.sequential_feature_selection_forward, fsm.sequential_feature_selection_backward,\
             fsm.maximally_predictive_minimally_demographic]
etas = [1,10,20,30]
feat_dictionary= {'hours-per-week':3,'capital-gain':10,'capital-loss':5,'education-num':5 ,'workclass':7,\
                  'education':16, 'relationship':6, 'occupation':14 }
 

thesis_param_grid = {'data':[adult_dataset],'features_cat': [features_cat], 'features_num':[features_num],\
                     'protected_attributes':[protected_attributes], 'p_att_description' : p_att_description,\
                     'target': [target], 'etas': [etas], 'feat_dictionary' : [feat_dictionary],\
                     'technique': technique, 'model': model}

my_expt = Experiment(fsm.main_exp_bf_func,thesis_param_grid)
batchname, successes ,fails = my_expt.run_batch(expt_name = 'binary_adult_data', name_func = name_function)



Index(['filtered_sex', 'workclass', 'education', 'hours-per-week',
       'capital-gain', 'capital-loss', 'education-num'],
      dtype='object')
['filtered_sex', 'workclass', 'education', 'hours-per-week', 'capital-gain', 'capital-loss', 'education-num', 'income-per-year']
Index(['filtered_sex', 'hours-per-week', 'capital-gain', 'capital-loss',
       'education-num'],
      dtype='object')
['filtered_sex', 'hours-per-week', 'capital-gain', 'capital-loss', 'education-num', 'income-per-year']
Index(['filtered_sex', 'hours-per-week', 'capital-gain', 'capital-loss',
       'education-num'],
      dtype='object')
['filtered_sex', 'hours-per-week', 'capital-gain', 'capital-loss', 'education-num', 'income-per-year']
Index(['filtered_sex', 'capital-gain', 'education-num', 'education',
       'capital-loss', 'hours-per-week'],
      dtype='object')
['filtered_sex', 'capital-gain', 'education-num', 'education', 'capital-loss', 'hours-per-week', 'income-per-year']
Index(['filtered_race', 'workc

In [19]:
my_results = ExperimentResult('results/'+ 'binary_adult_data2022_10_29_04_52_16_411705/')
my_results.get_result_names()

['adult_maximally_predictive_minimally_demographic_base_classifier_age_bins',
 'adult_maximally_predictive_minimally_demographic_base_classifier_marital-status',
 'adult_maximally_predictive_minimally_demographic_base_classifier_race',
 'adult_maximally_predictive_minimally_demographic_base_classifier_sex',
 'adult_maximally_predictive_minimally_demographic_fair_classifier_age_bins',
 'adult_maximally_predictive_minimally_demographic_fair_classifier_marital-status',
 'adult_maximally_predictive_minimally_demographic_fair_classifier_race',
 'adult_maximally_predictive_minimally_demographic_fair_classifier_sex',
 'adult_sequential_feature_selection_backward_base_classifier_age_bins',
 'adult_sequential_feature_selection_backward_base_classifier_marital-status',
 'adult_sequential_feature_selection_backward_base_classifier_race',
 'adult_sequential_feature_selection_backward_base_classifier_sex',
 'adult_sequential_feature_selection_backward_fair_classifier_age_bins',
 'adult_sequential_f

In [20]:
binary_adult_results = my_results.stack_results()

In [21]:
binary_adult_results

,data,features_cat,features_num,protected_attributes,p_att_description,target,etas,feat_dictionary,technique,model,dir_name,model_type,disparate_impact,statistical_parity_difference,equal_opportunity_difference,accuracy,eta
0,<__main__.NamedDataset object at 0x7fcc3d45b3d0>,"['workclass', 'relationship', 'occupation', 'e...","['hours-per-week', 'capital-gain', 'capital-lo...","['sex', 'race', 'age_bins', 'marital-status']","('age_bins', ['(38.9, 46.2]', '(46.2, 53.5]', ...",income-per-year,"[1, 10, 20, 30]","{'hours-per-week': 3, 'capital-gain': 10, 'cap...",<function maximally_predictive_minimally_demog...,<function base_classifier at 0x7fcc3d48a440>,adult_maximally_predictive_minimally_demograph...,base_model,0.480229,-0.074947,-0.046280,0.798010,NaN
0,<__main__.NamedDataset object at 0x7fcc3d45b3d0>,"['workclass', 'relationship', 'occupation', 'e...","['hours-per-week', 'capital-gain', 'capital-lo...","['sex', 'race', 'age_bins', 'marital-status']","('marital-status', ['Married-civ-spouse'])",income-per-year,"[1, 10, 20, 30]","{'hours-per-week': 3, 'capital-gain': 10, 'cap...",<function maximally_predictive_minimally_demog...,<function base_classifier at 0x7fcc3d48a440>,adult_maximally_predictive_minimally_demograph...,base_model,0.375549,-0.093383,0.092175,0.798010,NaN
0,<__main__.NamedDataset object at 0x7fcc3d45b3d0>,"['workclass', 'relationship', 'occupation', 'e...","['hours-per-week', 'capital-gain', 'capital-lo...","['sex', 'race', 'age_bins', 'marital-status']","('race', ['White'])",income-per-year,"[1, 10, 20, 30]","{'hours-per-week': 3, 'capital-gain': 10, 'cap...",<function maximally_predictive_minimally_demog...,<function base_classifier at 0x7fcc3d48a440>,adult_maximally_predictive_minimally_demograph...,base_model,0.595248,-0.061716,-0.059123,0.814151,NaN
0,<__main__.NamedDataset object at 0x7fcc3d45b3d0>,"['workclass', 'relationship', 'occupation', 'e...","['hours-per-week', 'capital-gain', 'capital-lo...","['sex', 'race', 'age_bins', 'marital-status']","('sex', [1])",income-per-year,"[1, 10, 20, 30]","{'hours-per-week': 3, 'capital-gain': 10, 'cap...",<function maximally_predictive_minimally_demog...,<function base_classifier at 0x7fcc3d48a440>,adult_maximally_predictive_minimally_demograph...,base_model,0.479924,-0.073731,-0.048759,0.798673,NaN
0,<__main__.NamedDataset object at 0x7fcc3d45b3d0>,"['workclass', 'relationship', 'occupation', 'e...","['hours-per-week', 'capital-gain', 'capital-lo...","['sex', 'race', 'age_bins', 'marital-status']","('age_bins', ['(38.9, 46.2]', '(46.2, 53.5]', ...",income-per-year,"[1, 10, 20, 30]","{'hours-per-week': 3, 'capital-gain': 10, 'cap...",<function maximally_predictive_minimally_demog...,<function fair_classifier at 0x7fcc3d48a4d0>,adult_maximally_predictive_minimally_demograph...,fair_model,0.238307,-0.165141,-0.185733,0.808948,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,<__main__.NamedDataset object at 0x7fcc3d45b3d0>,"['workclass', 'relationship', 'occupation', 'e...","['hours-per-week', 'capital-gain', 'capital-lo...","['sex', 'race', 'age_bins', 'marital-status']","('race', ['White'])",income-per-year,"[1, 10, 20, 30]","{'hours-per-week': 3, 'capital-gain': 10, 'cap...",<function xy_greater_xa at 0x7fcc3d489cf0>,<function fair_classifier at 0x7fcc3d48a4d0>,adult_xy_greater_xa_fair_classifier_race,fair_model,0.469084,-0.076236,-0.090932,0.816540,30.0
0,<__main__.NamedDataset object at 0x7fcc3d45b3d0>,"['workclass', 'relationship', 'occupation', 'e...","['hours-per-week', 'capital-gain', 'capital-lo...","['sex', 'race', 'age_bins', 'marital-status']","('sex', [1])",income-per-year,"[1, 10, 20, 30]","{'hours-per-week': 3, 'capital-gain': 10, 'cap...",<function xy_greater_xa at 0x7fcc3d489cf0>,<function fair_classifier at 0x7fcc3d48a4d0>,adult_xy_greater_xa_fair_classifier_sex,fair_model,0.195530,-0.136303,-0.180084,0.810865,1.0
1,<__main__.NamedDataset object at 0x7fcc3d45b3d0>,"['workclass', 'relationship', 'occupation', 'e...","[

In [22]:
binary_adult_results.to_csv('binary_adult_results.csv')

In [ ]:
print('world')